# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [2]:
# Import libraries
import numpy as np
import pandas as pd

In [3]:
# Importing the dataset
pokemon = pd.read_csv('Pokemon.csv')

From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [4]:
# First I will convert all column names to lowercase and replace spaces for "_"
def lowerand_(df):

    df.columns = df.columns.str.lower()
    df.rename(columns=lambda x: x.replace(' ', '_'), inplace=True)
    return df

In [5]:
lowerand_(pokemon)

,#,name,type_1,type_2,total,hp,attack,defense,sp._atk,sp._def,speed,generation,legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


In [6]:
# Also I want to delete all "." from the column names
pokemon.rename(columns=lambda x: x.replace('.', ''), inplace=True)

In [7]:
pokemon.columns

Index(['#', 'name', 'type_1', 'type_2', 'total', 'hp', 'attack', 'defense',
       'sp_atk', 'sp_def', 'speed', 'generation', 'legendary'],
      dtype='object')

In [8]:
pokemon['total_prediction'] = pokemon['hp']+pokemon['attack']+pokemon['defense']+pokemon['sp_atk']+pokemon['sp_def']+pokemon['speed']

In [9]:
pokemon.head(10)

,#,name,type_1,type_2,total,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary,total_prediction
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,318
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,405
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,525
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False,625
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False,309
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False,405
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False,534
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False,634
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False,634
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False,314


In [10]:
# verify if the expectations were met
verifying_pred = list(pokemon['total_prediction'] == pokemon['total'])
verifying_pred_df = pd.DataFrame(verifying_pred)
verifying_pred_df.value_counts()

True    800
Name: count, dtype: int64

## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [11]:
# import libraries
from sklearn.preprocessing import OneHotEncoder

In [12]:
pokemon_cat = pokemon[['type_1','type_2']]

In [13]:
pokemon.nunique()

#                   721
name                800
type_1               18
type_2               18
total               200
hp                   94
attack              111
defense             103
sp_atk              105
sp_def               92
speed               108
generation            6
legendary             2
total_prediction    200
dtype: int64

In [14]:
# onehotencode

encoder = OneHotEncoder().fit(pokemon_cat)
print(encoder.categories_)
encoded = encoder.transform(pokemon_cat).toarray()
print(encoded)
onehot_encoded = pd.DataFrame(encoded,columns=['Bug', 'Dark', 'Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire',
       'Flying', 'Ghost', 'Grass', 'Ground', 'Ice', 'Normal', 'Poison',
       'Psychic', 'Rock', 'Steel', 'Water','Bug_2', 'Dark_2', 'Dragon_2', 'Electric_2', 'Fairy_2', 'Fighting_2', 'Fire_2', 'Flying_2', 'Ghost_2', 'Grass_2', 'Ground_2', 'Ice_2', 'Normal_2', 'Poison_2', 'Psychic_2', 'Rock_2', 'Steel_2', 'Water_2', 'nan'])
onehot_encoded.head(20)


[array(['Bug', 'Dark', 'Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire',
       'Flying', 'Ghost', 'Grass', 'Ground', 'Ice', 'Normal', 'Poison',
       'Psychic', 'Rock', 'Steel', 'Water'], dtype=object), array(['Bug', 'Dark', 'Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire',
       'Flying', 'Ghost', 'Grass', 'Ground', 'Ice', 'Normal', 'Poison',
       'Psychic', 'Rock', 'Steel', 'Water', nan], dtype=object)]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,...,Grass_2,Ground_2,Ice_2,Normal_2,Poison_2,Psychic_2,Rock_2,Steel_2,Water_2,nan
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [15]:
# drop the nan column 
onehot_encoded = onehot_encoded.drop(['nan'], axis=1)

In [16]:
onehot_encoded['Bug']=onehot_encoded['Bug']+onehot_encoded['Bug_2']

In [17]:
onehot_encoded['Dark'] = onehot_encoded['Dark']+ onehot_encoded['Dark_2']
onehot_encoded['Electric'] = onehot_encoded['Electric'] + onehot_encoded['Electric_2']
onehot_encoded['Dragon'] = onehot_encoded['Dragon'] + onehot_encoded['Dragon_2']
onehot_encoded['Fairy'] = onehot_encoded['Fairy'] + onehot_encoded['Fairy_2']
onehot_encoded['Fighting'] = onehot_encoded['Fighting'] + onehot_encoded['Fighting_2']
onehot_encoded['Fire'] = onehot_encoded['Fire'] + onehot_encoded['Fire_2']
onehot_encoded['Flying'] = onehot_encoded['Flying'] + onehot_encoded['Flying_2']
onehot_encoded['Ghost'] = onehot_encoded['Ghost'] + onehot_encoded['Ghost_2']
onehot_encoded['Grass'] = onehot_encoded['Grass'] + onehot_encoded['Grass_2']
onehot_encoded['Ground'] = onehot_encoded['Ground'] + onehot_encoded['Ground_2']
onehot_encoded['Ice'] = onehot_encoded['Ice'] + onehot_encoded['Ice_2']
onehot_encoded['Normal'] = onehot_encoded['Normal'] + onehot_encoded['Normal_2']
onehot_encoded['Poison'] = onehot_encoded['Poison'] + onehot_encoded['Poison_2']
onehot_encoded['Psychic'] = onehot_encoded['Psychic'] + onehot_encoded['Psychic']
onehot_encoded['Rock'] = onehot_encoded['Rock'] + onehot_encoded['Rock_2']
onehot_encoded['Steel'] = onehot_encoded['Steel'] + onehot_encoded['Steel_2']
onehot_encoded['Water'] = onehot_encoded['Water'] + onehot_encoded['Water_2']

In [18]:
onehot_encoded_combined_cols = onehot_encoded[['Bug', 'Dark', 'Dragon', 'Electric', 'Fairy', 'Fighting', 'Fire',
       'Flying', 'Ghost', 'Grass', 'Ground', 'Ice', 'Normal', 'Poison',
       'Psychic', 'Rock', 'Steel', 'Water']]
onehot_encoded_combined_cols

,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
796,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
797,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
798,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0


In [19]:
onehot_encoded_combined_cols.dtypes

Bug         float64
Dark        float64
Dragon      float64
Electric    float64
Fairy       float64
Fighting    float64
Fire        float64
Flying      float64
Ghost       float64
Grass       float64
Ground      float64
Ice         float64
Normal      float64
Poison      float64
Psychic     float64
Rock        float64
Steel       float64
Water       float64
dtype: object

In [20]:
columns_encoded = list(onehot_encoded_combined_cols.columns)

In [21]:
for col in columns_encoded:
    onehot_encoded_combined_cols[col] = onehot_encoded_combined_cols[col].astype(int)

C:\Users\pablo\AppData\Local\Temp\ipykernel_17420\1526544949.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onehot_encoded_combined_cols[col] = onehot_encoded_combined_cols[col].astype(int)
C:\Users\pablo\AppData\Local\Temp\ipykernel_17420\1526544949.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onehot_encoded_combined_cols[col] = onehot_encoded_combined_cols[col].astype(int)
C:\Users\pablo\AppData\Local\Temp\ipykernel_17420\1526544949.py:2: SettingWithCopyWarning: 
A value is trying to be set on

In [22]:
onehot_encoded_combined_cols.dtypes

Bug         int32
Dark        int32
Dragon      int32
Electric    int32
Fairy       int32
Fighting    int32
Fire        int32
Flying      int32
Ghost       int32
Grass       int32
Ground      int32
Ice         int32
Normal      int32
Poison      int32
Psychic     int32
Rock        int32
Steel       int32
Water       int32
dtype: object

In [23]:
onehot_encoded_combined_cols.head()

,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [24]:
# I check that there is still only 0 and 1 values for each column
onehot_encoded_combined_cols.nunique()

Bug         2
Dark        2
Dragon      2
Electric    2
Fairy       2
Fighting    2
Fire        2
Flying      2
Ghost       2
Grass       2
Ground      2
Ice         2
Normal      2
Poison      2
Psychic     2
Rock        2
Steel       2
Water       2
dtype: int64

## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [26]:
onehot_encoded_combined_cols['Total']= pokemon['total']

C:\Users\pablo\AppData\Local\Temp\ipykernel_17420\3099988054.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  onehot_encoded_combined_cols['Total']= pokemon['total']


In [28]:
onehot_encoded_combined_cols.corr()["Total"].sort_values(ascending = False)

Total       1.000000
Dragon      0.229705
Steel       0.109703
Psychic     0.094364
Fire        0.078726
Fighting    0.077786
Ice         0.060248
Flying      0.059383
Dark        0.056154
Rock        0.032731
Electric    0.020971
Ground      0.015060
Ghost       0.003641
Water      -0.021665
Fairy      -0.036698
Grass      -0.052592
Poison     -0.090441
Normal     -0.105331
Bug        -0.145781
Name: Total, dtype: float64

In [ ]:
# The pokemon type that is most likely to have the highest 'Total' is 'Dragon'

# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [ ]:
# I will choose 'Dragon' and 'Steel'